In [ ]:
# Function to generate and analyze filtrations for a specified number of PCM instances
def generate_filtration_PCM(num_instances=5):
    create_new_db = True
    # Initialize databases for CL(4) and CL(n) filtrations
    db4 = CLFiltrationDB(filepath_generator(dirname=DIR_NAME,filename=f"{DB_NAME}_4", extension='db',overwrite=not create_new_db),create_new_db=create_new_db)
    dbn = CLFiltrationDB(filepath_generator(dirname=DIR_NAME,filename=f"{DB_NAME}_n", extension='db',overwrite=not create_new_db),create_new_db=create_new_db)
    
    # Extract parameters from configuration for PCM generation
    ladder_len_min, ladder_len_max = POINT_CLOUD_PARAMS['ladder_len_min'], POINT_CLOUD_PARAMS['ladder_len_max']
    pts_min, pts_max = POINT_CLOUD_PARAMS['pts_min'], POINT_CLOUD_PARAMS['pts_max']
    enable_multi_processing, num_cores, verbose = POINT_CLOUD_PARAMS['enable_multi_processing'], POINT_CLOUD_PARAMS['num_cores'], POINT_CLOUD_PARAMS['verbose']
    
    i = 0
    while i < num_instances:  # Loop through the specified number of instances
        num_pts = random.randint(pts_min, pts_max)  # Determine the number of points for this model
        # Generate a PCM with random filtration
        result = RandomFiltrationPointCloudModel(num_pts=num_pts, ladder_length_min=ladder_len_min, ladder_length_max=ladder_len_max, enable_multi_processing=enable_multi_processing, num_cores=num_cores, verbose=verbose)
        i += 1
        print(i)  # Logging the current instance number
        
        # Process and store CL(4) filtration results, if present
        if filtration := result.output["CL(4)"]:
            # Identify non-intervals in the decomposition
            non_intervals = {k:v for k,v in filtration.info['decomp'].items() if k[0] == 'N'}
            if non_intervals:
                print(f'non_intervals found @ {i}', flush=True)
                # Send a message indicating non-intervals were found
                send_finish_message(f"non_intervals {non_intervals} found @ {i}, point cloud model, homology dimension: {filtration.info['homology_dim']}.")
            db4.add_filtration(filtration)  # Add filtration results to the database
        
        # Process and store CL(n) filtration results, if present
        if filtration := result.output["CL(n)"]:
            # Check for non-intervals in CL(n) results
            non_intervals = (pd.read_csv(StringIO(filtration.info['lines']), index_col=0).multiplicity < 0).any()
            if non_intervals:
                # Send a message indicating non-intervals were found in CL(n) results
                send_finish_message(f"non_intervals {filtration.info['lines']} found @ {i}, point cloud model, homology dimension: {filtration.info['homology_dim']}, ladder_length: {filtration.ladder_length}.")
            dbn.add_filtration(filtration)  # Add CL(n) filtration results to the database
